<div style="text-align: center"><h1 style="text-decoration: underline;">DSML Project</h1></div>



This is the official Notebook of the DSML Project from Marc Rennefort, Killian Lipinsky, Timo Hagelberg, Jan Behrendt-Emden and Paul Severin. In order to create this Project we used the following dataset: https://data.cityofchicago.org/Transportation/Transportation-Network-Providers-Trips-2023-2024-/n26f-ihde/about_data
<h4>1. Description</h4>
The goal of this Project is to predict the tip for ride-hailing in Chicago based on the travelled time and distance as well as the spended money, the wheater and the information if the customer matched to a shared trip or not. 


In [25]:
#Note all your imports here
import pandas as pd